# Obtención de datos desde la API del Banco Mundial

In [ ]:
import requests
import pandas as pd

## Extracción de los datos de la variable objetivo: Esperanza de Vida al Nacer

Según David, Rodríguez (2015) los factores que inciden sobre la Esperanza de Vida al Nacer dependen en gran medida de los ingresos per capita del país. Por ello vamos a extraer datos de paises que pertenezcan a cada una de las 4 categorías: Ingresos Bajos, Ingresos Medio - Bajos, Ingresos Medio - Altos e Ingresos Altos. Todo ello según la clasificación del Banco Mundial

Aqui vemos los codigos para cada categoría que sera el argumento para la extraccion de los paises

In [53]:
url = 'http://api.worldbank.org/v2/incomelevel' #Aqui vemos la lista de niveles de Ingreso y sus respectivos códigos
args = {"format":"json", "prefix":"Getdata"}
dic_Income_Level = requests.get(url, params=args)
datos = dic_Income_Level.json()[1]
data = pd.json_normalize(datos)
data[['value', 'id']]

,value,id
0,High income,HIC
1,Not classified,INX
2,Low income,LIC
3,Lower middle income,LMC
4,Low & middle income,LMY
5,Middle income,MIC
6,Upper middle income,UMC


Ahora vamos a tomar una muestra de 10 paises de cada categoría y vamos guardando en una lista los id para luego a partir de esa lista ir extrayendo los datos para cada indicador.

Nivel de Ingresos: Alto

In [54]:
incomes = ['HIC', 'UMC', 'LMC', 'LIC']
trabajo = pd.DataFrame()
for income in incomes:
    url = 'http://api.worldbank.org/v2/country' #Aqui vemos la lista de niveles de Ingreso y sus respectivos códigos
    args = {"format":"json", "prefix":"Getdata", "incomelevel":income}
    dic_Income_Level = requests.get(url, params=args)
    datos = dic_Income_Level.json()[1]
    data = pd.json_normalize(datos)
    data = data[['name', 'id', 'region.value', 'incomeLevel.value']]
    trabajo = pd.concat([trabajo, data], ignore_index=True)


In [55]:
trabajo

,name,id,region.value,incomeLevel.value
0,Aruba,ABW,Latin America & Caribbean,High income
1,Andorra,AND,Europe & Central Asia,High income
2,United Arab Emirates,ARE,Middle East & North Africa,High income
3,Antigua and Barbuda,ATG,Latin America & Caribbean,High income
4,Australia,AUS,East Asia & Pacific,High income
...,...,...,...,...
173,Chad,TCD,Sub-Saharan Africa,Low income
174,Togo,TGO,Sub-Saharan Africa,Low income
175,Uganda,UGA,Sub-Saharan Africa,Low income
176,"Yemen, Rep.",YEM,Middle East & North Africa,Low income


Ahora vamos a crear una función que reciba la API, la lista, el periodo temporal y un indicador y lo extraiga de la API

In [ ]:
def extraer (api_url,lista, indicador,tiempo):
    #creamos el argumento paises
    lista3 = lista.copy()
    b = lista3[0]
    lista3.pop(0)
    for a in lista3:
        b = b+';'+a #la API devuelve los datos de varios paises pasando el argumento: AVW;AND;ETH...
    #lista3 = ";".join(lista3) otra forma
    path = f'/country/{b}/indicator/{indicador}'
    url = api_url + path
    args = {"format":"json", "prefix":"Getdata","date":tiempo,"per_page":2000}
    s = requests.get(url, params=args)
    salida = pd.json_normalize(s.json()[1])
    return(salida[['countryiso3code','date','value']])

ahora vamos a implementar la función

In [56]:
lista_ids= list(trabajo['id'])

url = 'http://api.worldbank.org/v2'
indicador = 'SP.POP.TOTL'
tiempo = '1991:2020'
df = extraer(api_url=url,lista=lista_ids,indicador=indicador,tiempo=tiempo)
df

http://api.worldbank.org/v2/country/ABW;AND;ARE;ATG;AUS;AUT;BEL;BHR;BHS;BMU;BRB;BRN;CAN;CHE;CHI;CHL;CUW;CYM;CYP;CZE;DEU;DNK;ESP;EST;FIN;FRA;FRO;GBR;GIB;GRC;GRL;GUM;HKG;HRV;HUN;IMN;IRL;ISL;ISR;ITA;JPN;KNA;KOR;KWT;LIE;LTU;LUX;LVA;MAC;MAF;ALB;ARG;ARM;ASM;AZE;BGR;BIH;BLR;BLZ;BRA;BWA;CHN;COL;CRI;CUB;DMA;DOM;ECU;FJI;GAB;GEO;GNQ;GRD;GTM;GUY;IRQ;JAM;JOR;KAZ;LBY;LCA;MDA;MDV;MEX;MHL;MKD;MNE;MUS;MYS;NAM;PER;PLW;PRY;RUS;SRB;SUR;THA;TKM;TON;TUR;AGO;BEN;BGD;BOL;BTN;CIV;CMR;COG;COM;CPV;DJI;DZA;EGY;FSM;GHA;HND;HTI;IDN;IND;IRN;KEN;KGZ;KHM;KIR;LAO;LBN;LKA;LSO;MAR;MMR;MNG;MRT;NGA;NIC;NPL;PAK;PHL;PNG;PSE;SEN;SLB;SLV;STP;SWZ;TJK;TLS;TUN;TZA;UKR;UZB;AFG;BDI;BFA;CAF;COD;ERI;ETH;GIN;GMB;GNB;LBR;MDG;MLI;MOZ;MWI;NER;PRK;RWA;SDN;SLE;SOM;SSD;SYR;TCD;TGO;UGA;YEM;ZMB/indicator/SP.POP.TOTL


,countryiso3code,date,value
0,ABW,2020,106766
1,ABW,2019,106310
2,ABW,2018,105846
3,ABW,2017,105361
4,ABW,2016,104865
...,...,...,...
995,CHE,2015,8282396
996,CHE,2014,8188649
997,CHE,2013,8089346
998,CHE,2012,7996861


Selección de un solo país bajo estudio. Si se eligen todos los países la API tarda un poco más de 30 segundos en responder una vez terminado el ciclo for. Y esa carga inicial podría molestar al usuario final.

In [ ]:
paises=pd.read_csv('../datasets/paises.csv')

seleccion = input("Ingrese un nombre de país: ")
try:
    pais = paises.loc[paises['country.value'].str.contains(seleccion, case=False),'countryiso3code'].values[0]
except:
    print("No se encontró el país, intente de nuevo")

In [ ]:
# Establecemos los parámetros que queremos acceder.
indicador = 'SP.DYN.LE00.IN'
#pais = 'all' # Está comentado porque lo recibe del bloque anterior
iyear = '1980'
fyear = '2022'

# Página de la api y path al recurso solicitado
api_url = ' http://api.worldbank.org/v2/es'
path = f'/country/{pais}/indicator/{indicador}'

url = api_url + path

# Creamos el diccionario con los parametros 
# para el método get
def consultar(pagina = 1):
    args= {
        "date":iyear+":"+fyear,
        'page':pagina,
        "per_page":1000,
        "format":"json",
        "prefix":"Getdata",
    }
    return requests.get(url,params=args).json()

dict_data = consultar()

In [ ]:
# La primera parte de la respuesta nos indica en 
# cuantas páginas de encuentra la información
paginas=dict_data[0]["pages"]
paginas # Retornamos el número de páginas

In [ ]:
# La segunda parte nos retorna una lista de diccionarios 
# con la información que queríamos
datos=dict_data[1]
datos

In [ ]:
# Agregamos los valores de las otras páginas, si existen a 
# nuestra lista de diccionarios
for pagina in range(2,paginas+1):
    datos.extend(consultar(pagina)[1])

In [ ]:
# Verifico que lo obtenido sea una lista de diccionarios
print(type(datos))
print(type(datos[1]))

In [ ]:
# Creo el DataFrame con todos los datos
data = pd.json_normalize(datos)

In [ ]:
# Visualizo información relevante unicamente
data[['date','value','country.value']]

In [ ]:
# Creo un dataset para trabajar offline
# data.to_csv('../datasets/esperanza_de_vida.csv')

# EDA

Esto se hace en la primera ejecución del código, cuando se corría con todos los países

In [ ]:
paises = data[['countryiso3code','country.value']].groupby('countryiso3code').first()

# Regreso el index creado por agrupar a una columna
paises.reset_index(inplace=True)

# Reemplazo todos los valores faltantes por NaN
paises.replace("", float("NaN"), inplace=True)

paises.dropna(inplace=True)

# Creo el dataset de países que me va a servir de selección
# paises.to_csv('../datasets/paises.csv')

Verifico los datos faltantes en la respuesta de la API en español. Son dós nombres de países o grupo de países, con el código AFW y AFE, para Africa ocidental y central y Africa del este y del sur, respectivamente. En resumen: Regiones de Africa. No sé si lo vamos a incluir

In [ ]:
data[data.countryiso3code.str.contains('AFE')]

## Búsqueda de indicadores

In [ ]:
url='http://api.worldbank.org/v2/es/indicator'
def consultar_indicadores(pagina=1):
    argumentos= {
        'page':pagina,
        'per_page':10000,
        'format':'json'
    }
    return requests.get(url,params=argumentos).json()

indicadores = consultar_indicadores()

# la primera parte de la respuesta 
# contiene información sobre las páginas
paginas = indicadores[0]["pages"]

# Reasigno la variable únicamente a los datos de mi interés 
# es una lista de diccionarios
indicadores = indicadores[1]

# recorro todás las páginas que tiene la respuesta 
# y lo voy anexando a la lista principal
for pagina in range(2, paginas+1):
    indicadores.extend(consultar_indicadores(pagina)[1])
indicators = pd.json_normalize(indicadores)

# Dev data para trabajo offline
# indicators.to_csv('../datasets/indicadores.csv')

interes = input("Indicador de interés: ")
indicators.loc[indicators.name.str.contains(interes,case=False),
     ['id','name','sourceNote']]#.to_csv('temp.csv')